In [4]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import numpy as np
import matplotlib.pyplot as plt
from squanch import *
% matplotlib inline

UsageError: Line magic function `%` not found.


In [ ]:
class Alice(Agent):
    '''Alice sends qubits to Bob using a shared Bell pair'''
    
    def distribute_bell_pair(self, a, b):
        # Create a Bell pair and send one particle to Bob
        H(a)
        CNOT(a, b)
        self.qsend(bob, b)

    def teleport(self, q, a):
        # Perform the teleportation
        CNOT(q, a)
        H(q)
        # Tell Bob whether to apply Pauli-X and -Z over classical channel
        bob_should_apply_x = a.measure() # if Bob should apply X
        bob_should_apply_z = q.measure() # if Bob should apply Z
        self.csend(bob, [bob_should_apply_x, bob_should_apply_z])

    def run(self):
        for qsystem in self.qstream:
            q, a, b = qsystem.qubits # q is state to teleport, a and b are Bell pair
            self.distribute_bell_pair(a, b)
            self.teleport(q, a)

In [ ]:
class Bob(Agent):
    '''Bob receives qubits from Alice and measures the results'''

    def run(self):
        measurement_results = []
        for _ in self.qstream:
            # Bob receives a qubit from Alice
            b = self.qrecv(alice) 
            # Bob receives classical instructions from alice
            should_apply_x, should_apply_z = self.crecv(alice)
            if should_apply_x: X(b)
            if should_apply_z: Z(b)
            # Measure the output state
            measurement_results.append(b.measure())
        # Put results in output object
        self.output(measurement_results)

In [ ]:
# Prepare the initial states
qstream = QStream(3,10) # 3 qubits per trial, 10 trials
states_to_teleport = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
for state, qsystem in zip(states_to_teleport, qstream):
    q = qsystem.qubit(0)
    if state == 1: X(q) # flip the qubits corresponding to 1 states

# Make and connect the agents
out = Agent.shared_output()
alice = Alice(qstream, out)
bob = Bob(qstream, out)
alice.qconnect(bob) # add a quantum channel
alice.cconnect(bob) # add a classical channel

# Run everything
alice.start()
bob.start()
alice.join()
bob.join()

print("Teleported states {}".format(states_to_teleport))
print("Received states   {}".format(out["Bob"]))

In [5]:
angles = np.linspace(0, 2 * np.pi, 50)  # RX angles to apply
num_trials = 250  # number of trials for each angle

# Prepare the initial states in the stream
qstream = QStream(3, len(angles) * num_trials)
for angle in angles:
    for _ in range(num_trials):
        q, _, _ = qstream.next().qubits
        RX(q, angle)

# Make the agents and connect with quantum and classical channels
out = Agent.shared_output()
alice = Alice(qstream, out = out)
bob = Bob(qstream, out = out)
alice.qconnect(bob)
alice.cconnect(bob)

# Run the simulation
Simulation(alice, bob).run()

# Plot the results
results = np.array(out["Bob"]).reshape((len(angles), num_trials))
observed = np.mean(results, axis = 1)
expected = np.sin(angles / 2) ** 2
plt.plot(angles, observed, label = 'Observed')
plt.plot(angles, expected, label = 'Expected')
plt.legend()
plt.xlabel("Rotation angle $\\theta$")
plt.ylabel("Fractional $\left | 1 \\right >$ population")
plt.show()

NameError: name 'Alice' is not defined